# Exercício prático 05 - Resolução de sistemas não lineares  
### Resolução das matrizes jacobianas - matrizes das derivadas parciais
### Método de newton para sistemas não Lineares



In [38]:
def precisao_maquina():
    """Função que calcula a precisão da máquina", o número positivo em aritmética de ponto flutuante 𝜀, tal que 1 + 𝜀 >1."""
    #passo 1
    A=1
    S=2
    #passo 2
    while(S>1):
        A=A/2
        S=1+A
    prec=2*A #passo 3
    
    return prec

# def MétodoNewton_NaoLineares(M_Jacobiana,precisao = precisao_maquina() ):


In [5]:
import numpy as np
import pandas as pd
import sympy as sym

def getMatrizJacobiana(F,n_variaveis):
    """Função que gera a matriz jacobiana de uma matriz de funções que recebe como parâmetro\n
    Parâmetros:\n
    F = matriz principal da qual se quer gerar a jacobiana\n
    Retorno:\n Matriz Jaconiana de F"""

    # pega a dimensao da matriz
    n = len(F)
    m = len(F[0])
    # gera as variáveis
    symbols = sym.symbols(f"x1:{n_variaveis+1}")
    matrizJacobiana=list()# vriável que guardará o retorno da função

    # print(n,m)
    # print(symbols)
    # percorre a matriz F
    for i in range(n):
        listaux=list()
        for j in range(m):
            f = F[i][j]
        
            # print(f"Função= {f}")

            derivadas_list =list()
            #calculo da derivada pra cada variável
            for variable in symbols:
                #calculo da derivada parcial
                derivative = f.diff(variable)
                # print(f"Derivada de {variable} :{derivative}")
                derivadas_list.append(derivative)
              
            
            listaux.append(derivadas_list)
        # print("Lista de derivadas parciais = ",listaux)
        matrizJacobiana.append(listaux)

    return matrizJacobiana


### Exemplo de matriz jacobiana

In [64]:

#dada as funções f de 1 e f2 e a matriz F
x1,x2 = sym.symbols("x1 x2")
f1 = x1**3 -3*x1*x2**2 +1
f2 = 3*x1**2*x2 - x2**3
F=[[f1],
   [f2]]

print("Matriz F(X)")
display(pd.DataFrame(F))
print("Matriz Jacobiana J(X)")
J = getMatrizJacobiana(F,2)
display(pd.DataFrame(J))

Matriz F(X)


,0
0,x1**3 - 3*x1*x2**2 + 1
1,3*x1**2*x2 - x2**3


Matriz Jacobiana J(X)


,0
0,"[3*x1**2 - 3*x2**2, -6*x1*x2]"
1,"[6*x1*x2, 3*x1**2 - 3*x2**2]"


#### Construindo a representação da matriz pra receber os valores de x

In [91]:
def F1(x):
    return np.matrix([
        [x[0]**3 -3*x[0]*x[1]**2+1],
        [3*x[0]**2*x[1] - x[1]**3]
    ])

def J1(x):
        return np.matrix([
         [3*x[0]**2 - 3*x[1]**2 , -6*x[0]*x[1]],
         [ 6*x[0]*x[1], 3*x[0]**2  - 3*x[1]**2 ]
    ])


#### Resolvendo forma 1

In [90]:
print(F1([1,2]))
print(J1([1,2]))

[[-10]
 [ -2]]
[[ -9 -12]
 [ 12  -9]]


### Exemplo 2

In [67]:
from sympy import sin,cos
f1 = x1**2 - cos(x1*x2) - 1
f2 = sin(x2) - 2*cos(x1)
F2_df=[[f1],
      [f2]]

print("Matriz F2(X)")
F2_df=pd.DataFrame(F2_df)
display(F2_df)
print("Matriz Jacobiana J2(X)")
J2_df = pd.DataFrame(getMatrizJacobiana(F,2))
display(J2_df)

Matriz F2(X)


,0
0,x1**2 - cos(x1*x2) - 1
1,sin(x2) - 2*cos(x1)


Matriz Jacobiana J2(X)


,0
0,"[3*x1**2 - 3*x2**2, -6*x1*x2]"
1,"[6*x1*x2, 3*x1**2 - 3*x2**2]"


In [68]:
import math
def F2(x):
    return np.matrix([ 
        [x[0]**2 - math.cos(x[0]*x[0]) - 1],
        [math.sin(x[0]) - 2*math.cos(x[0])]])

def J2(x):
        return np.matrix([
        [ 2*x[0] + x[1]*math.sin(x[0]*x[1]),x[0]*math.sin(x[0]*x[1]) ],
        [2*math.sin(x[0]) , math.cos(x[1])]
          ])


#### Exemplo 3 - livro(exemplo 5)

In [71]:

F3_df=[[x1 + x2 - 3] ,
        [x1**2 + x2**2 -9]]

print("Matriz F3(X)")
display(pd.DataFrame(F3_df))
print("Matriz Jacobiana J3(X)")
J3_df = pd.DataFrame(getMatrizJacobiana(F3_df,2))
display(J3_df)

Matriz F3(X)


,0
0,x1 + x2 - 3
1,x1**2 + x2**2 - 9


Matriz Jacobiana J3(X)


,0
0,"[1, 1]"
1,"[2*x1, 2*x2]"


In [72]:
def F3(x):  
    return np.matrix([
    x[0] + x[1] - 3 ,
    x[0]**2 + x[1]**2 -9])
 
def J3(x):  
    return np.matrix([
    [1 , 1],
    [2*x[0],2*x[1]]])


## Método de newton para sistemsa não lineares

#### Eliminação de Gauss

In [41]:
import numpy as np
import pandas as pd

# Resolve o sistema triangular superior.
def SistemaTriangularSuperior(Ax,b):
  """Função que resolve um sistema linear triangular superior do tipo  𝐴𝑥 = b que recebe como parâmetro """
  n = len(b) #tamanho do vetor b
  x= [b[i]/Ax[i][i] if i == 0 else None for i in range(0,len(b))]

  x[n-1] = b[n-1]/Ax[n-1][n-1]
  iter = 0
  for i in list(range(n-1,0,-1)):
    iter+=1
    s = 0
    for j in list(range(i+1,n+1)):
      s = s + Ax[i-1][j-1]*x[j-1]

    x[i-1] = (b[i-1]-s)/(Ax[i-1][i-1])

  return [x,iter]

def EliminacaoDeGauss(A,b):
  """Função que resolve um sistema linear do tipo Ax = b que recebe como parâmetro
     Return : retorna o vetor x resultante"""
  
  # ELIMINAÇÃO DE GAUSS
  n = len(b)

  # Calculo dos pivos.
  for k in range(1,n+1):
    # PIVOTEAMENTO PARCIAL
    for i in range(k+1, len(A)+1):
      if (abs(A[i-1][k-1]) > abs(A[k-1][k-1])):
        [A[k-1], A[i-1]] = [A[i-1], A[k-1]]
        [b[k-1], b[i-1]] = [b[i-1], b[k-1]]

    # Calculo dos multiplicadores.
    for i in list(range(k+1,n+1)):
      m = A[i-1][k-1]/A[k-1][k-1]
      A[i-1][k-1] = 0
      
      # Atualizar demais valores da linha
      for j in list(range(k+1,n+1)):
        A[i-1][j-1] = A[i-1][j-1]-m*A[k-1][j-1]
      b[i-1] = b[i-1] - m*b[k-1]

  # print("depois da eliminação\n Ax =  ",Ax2,"\nb = ",b2)
  x = SistemaTriangularSuperior(A,b)
  return(x)

#### Algoritmo de newton pra sistemas não lineares

In [96]:
def Newton_N_Lineares(F,J,x0,e1=precisao_maquina(),e2=precisao_maquina(),it_max=10000,nw_modificado=False):  
    verifica_precisao =np.max(F(x0)) 

    if verifica_precisao < e1:
        return(x0,0)
    xk=x0
    # print("Chute Inicial : ",F(x0))
    k=0

    while(verifica_precisao>e2 and k<it_max):
        # print(f"\n{k}\n")
        # print(f"J(x{k}) = ",J(xk).tolist())
        # print(f"-F(x{k})",(-F(xk)[0]).tolist()[0])

        s= np.matrix(EliminacaoDeGauss(J(x0).tolist(),(-F(xk)[0]).tolist()[0])[0]) if nw_modificado else np.matrix(EliminacaoDeGauss(J(xk).tolist(),(-F(xk)[0]).tolist()[0])[0])

        # print("s = ",s)
        xk=np.matrix(xk)
        xkp1=xk+s # calcula 0 x^k+1
        # print(f"x{k+1} = {xkp1}")

        if abs(np.max(np.abs(xkp1-xk))) < e2:
            # print("menor")
            return xkp1.tolist()[0],k
        
        # print(f"{np.max(np.abs(xkp1-xk))}> e2")
        k+=1
        xk=xkp1.tolist()[0] # atualiza o xk pra ser xk+1
        # print(xk)
        verifica_precisao=np.max(F(xk))

    return xk,k

### Resolução do exemplo 3

In [97]:
print("Matriz F3(X)")
display(pd.DataFrame(F3_df))
print("Matriz Jacobiana J3(X)")
display(J3_df)
R1=Newton_N_Lineares(F3,J3,[1,5])
print(R1)
print(f" X = {R1[0]} com {R1[1]} iterações")

Matriz F3(X)


,0
0,x1 + x2 - 3
1,x1**2 + x2**2 - 9


Matriz Jacobiana J3(X)


,0
0,"[1, 1]"
1,"[2*x1, 2*x2]"


([1.1567124056158726e-16, 3.0], 6)
 X = [1.1567124056158726e-16, 3.0] com 6 iterações


In [98]:
print("Matriz F2(X)")
display(pd.DataFrame(F2_df))
print("Matriz Jacobiana J3(X)")
display(J2_df)
R1=Newton_N_Lineares(F2,J2,[1.5,0.5])
print(R1)
print(f" X = {R1[0]} com {R1[1]} iterações")

Matriz F2(X)


,0
0,x1**2 - cos(x1*x2) - 1
1,sin(x2) - 2*cos(x1)


Matriz Jacobiana J3(X)


,0
0,"[3*x1**2 - 3*x2**2, -6*x1*x2]"
1,"[6*x1*x2, 3*x1**2 - 3*x2**2]"


([0.9378104862657436, -0.06218951373425641], 1)
 X = [0.9378104862657436, -0.06218951373425641] com 1 iterações


### Exemplo 6 - livro

In [102]:
def F6(x):  
    return np.matrix([
    x[0] + x[1] - 3 ,
    x[0]**2 + x[1]**2 -9])
 
def J6(x):  
    return np.matrix([
    [1 , 1],
    [2*x[0],2*x[1]]
    ])

R=Newton_N_Lineares(F6,J6,[1,5],nw_modificado = True)
print(f" X = {R[0]} com {R[1]} iterações")

 X = [0.0, 3.0] com 26 iterações
